In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn")
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

from nnet import ReadDataset, Net
import os
import sys

from sklearn.metrics import roc_auc_score


def evaluate_auc(model, data, label):
    return roc_auc_score(label.detach().numpy(), model(data.float()).detach().numpy())



In [2]:
csv_file = os.path.join(sys.path[0], "train.csv")


In [3]:
# Read data
dataset = ReadDataset(csv_file)


(212662, 77)


In [4]:
trainloader = DataLoader(dataset)

In [5]:
# Use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Neural Network
nnet = Net(dataset.__shape__()).to(device)

nnet.load_state_dict(torch.load("output/weights80.pt"))


<All keys matched successfully>

## Train

In [6]:
a = torch.tensor(dataset.X.values)
a = a.to(device).float()



a = nnet.forward(a)


a = a.detach().numpy()

b = dataset.y.values

roc_auc_score(b,a)

0.9228079378381548

## Test

In [7]:

# Read data
dataset = ReadDataset(csv_file,for_test=True)



(212662, 77)
(141776, 77)


In [8]:
dataset.X.shape

(141776, 77)

In [9]:
a = torch.tensor(dataset.X)
a = a.to(device).float()

In [10]:
a = nnet.forward(a)


a = a.detach().numpy()

In [11]:
a

array([1.5080692e-01, 1.4530003e-05, 2.7099580e-01, ..., 5.2201521e-01,
       9.7979522e-01, 3.2566030e-02], dtype=float32)

In [12]:

test = pd.read_csv("test.csv", index_col="Id")


In [13]:
test['Predicted'] = a

In [14]:
test[['Predicted']].to_csv('submissions/nn_1.csv')